# Quantum counting

To successfully run Grover's algorithm we need to know the exact number of solutions in order to determine the required number of iterations. However, this is sometimes not the case and an algorithm for determination of the number of solutions is required. Here we show how to determine the number of solutions using the Quantum phase estimation algorithm.

In [1]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Characterization;
open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;

Here we define a simple oracle in order to test our code. The oracle defines a function $f(x) = 1$ only when $x = 0\dots 0$ and $x = 1\dots 1$.

In [2]:
// oracle f(x) = 1 if x = 00..00, 11..11
operation Oracle(groverQubits : Qubit[]) : Unit is Adj+Ctl {
    Controlled X(Most(groverQubits), Tail(groverQubits));
    within {
        ApplyToEachA(X, Most(groverQubits));
    }
    apply {
        Controlled X(Most(groverQubits), Tail(groverQubits));
    }
}

Oracle

In [3]:
// Grover diffusion operator
operation GroverDiffusion(groverQubits : Qubit[]) : Unit is Adj+Ctl {
    within {
        ApplyToEachA(H, Most(groverQubits));
        ApplyToEachA(X, Most(groverQubits));
    } apply {
        Controlled Z(Most(Most(groverQubits)), Tail(Most(groverQubits)));
    }
}

GroverDiffusion

In [24]:
// Circuit performing a single Grover iteration
operation GroverIteration(groverQubits : Qubit[]): Unit is Adj+Ctl {
    Oracle(groverQubits);
    GroverDiffusion(groverQubits);
}

GroverIteration

In [5]:
operation GroverPow(power: Int, groverQubits : Qubit[]): Unit is Adj+Ctl {
    for i in 1 .. power {
        GroverIteration(groverQubits);
    }
}

GroverPow

In [25]:
// Circuit implementing the Quantum phase estimation algorithm
operation QuantumCounting(groverQubits : Qubit[], targetQubits : Qubit[]) : Unit is Adj+Ctl {
    let oracle = DiscreteOracle(GroverPow);
    QuantumPhaseEstimation(oracle, groverQubits, BigEndian(targetQubits));
}

QuantumCounting

In [12]:
operation MeasureTarget() : Int {
    use groverQubits = Qubit[6];
    use targetQubits = Qubit[5];
    
    QuantumCounting(groverQubits, targetQubits);
    
    let a = BoolArrayAsInt(Reversed(ResultArrayAsBoolArray(MultiM(targetQubits))));
    ResetAll(groverQubits);
    ResetAll(targetQubits);
    
    let theta = 2.0 * PI() * IntAsDouble(a) / IntAsDouble(2 ^ Length(targetQubits));
    let numSolutions= PowD(Cos(theta / 2.0), 2.0) * IntAsDouble(2 ^ (Length(groverQubits) - 1));
    Message($"a: {a}");
    return Round(numSolutions);
}

MeasureTarget

In [23]:
%simulate MeasureTarget

a: 16


0

In [14]:
%estimate MeasureTarget

Metric,Sum,Max
CNOT,7818,7818
QubitClifford,2564,2564
R,9,9
Measure,16,16
T,5464,5464
Depth,3554,3554
Width,15,15
QubitCount,15,15
BorrowedWidth,0,0
